# Analysis of Cancer Biopsy Image Data

This notebook presents the analysis of pandas databases, each containing PyFibre output data for multiple biopsy images. The databases are labelled according to the directories that they are located in, for example "Normal", "In Situ", "Misto" and "Solido - Invasor". 

The notebook performs feature selection based on Linear Discriminant Analysis (LDA) as well as Principle Component Analysis (PCA) on the image metrics of each image.

To begin with we load in all the libraries that we will need, including pandas database (http://pandas.pydata.org/), and scikit-learn machine learning (https://scikit-learn.org/stable/) packages. Both are standard data handling packages for Python.

In [2]:
import sys, os

import pandas as pd
from pandas.plotting import scatter_matrix

import numpy as np
from scipy import linalg

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFECV, f_classif
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import (
    auc, roc_curve, confusion_matrix, accuracy_score, classification_report)

Below we define a useful plotting function for a viewing the first two dimensions of a multi-dimensional array as a scatter plot coloured by labels. The size of the points can be determined by the probability of each assigned label.

In [ ]:
from analysis_tools import scatter, load_databases

We load in the databases for Normal, In Situ, Misto and Solido - Invasor labelled images. We can change the extension on each database file to load in just the fibre or cell segment databases. At the moment, we only load in the global image metrics.

In [ ]:
current_dir = os.getcwd()

# Extension options to load each type of PyFibre database
extensions = ['_global', '_fibre', '_cell', '_network']

# Enter in the names of directories here
directories = []
data_directories = [
    os.path.join(current_dir, group) + '/' for group in directories
]

filename = 'pyfibre_database'
database = load_databases(filename + extensions[0], data_directories)

We tidy up the databases to remove any entries with missing values . For cell and fibre databases, we only retain the 4 largest regions for each image.

In [ ]:
database_cut = database.copy()
groups = np.unique(database['Label'])

try: 
    database_cut = database_cut.where(database_cut['ID'] <= 0)
except: 
    pass

try: 
    database_cut = database_cut.where(database_cut['Cell Segment Area'] >= 200)
except: 
    pass

try: 
    database_cut = database_cut.where(database_cut['Fibre Segment Area'] >= 200)
except: 
    pass

try: 
    database_cut = database_cut.where(database_cut['No. Fibres'] >= 10)
except: 
    pass
        
# For now we remove the 'ID' entries for simplicity
try: 
    database_cut = database_cut.drop(['File', 'ID'], axis=1)
except: 
    database_cut = database_cut.drop(['File'], axis=1)

database_cut = database_cut.replace(0, np.nan)
database_cut = database_cut.replace(np.inf, np.nan)
database_cut = database_cut.dropna()

# Map each seperate group name to the new database
database_cut['Group'] = database_cut['Label'].map(
    {float(index + 1): value for index, value in enumerate(groups)})
database_cut['Group'] = pd.Categorical(
    database_cut['Group'], categories=groups, ordered=True)

# Display an excert from the total database
database_cut.head()

Next we create a normalised database where each metric ranges between 0 and 1.

In [ ]:
columns = list(database_cut.columns)
columns.remove('Label')
columns.remove('Group')

df_norm = database_cut.copy()
x = df_norm[columns].values

# StandardScalar scales all values between -1 to 1
x_scaled = StandardScaler().fit_transform(x)
# QuantileTransformer enforces a Gaussian distribution of variables
x_scaled = QuantileTransformer(output_distribution='normal').fit_transform(x_scaled)

df_norm[columns] = x_scaled
df_norm.head()

We can view boxplots of each metric corresponding to each labelled group. These are much easier to display side-by-side if they are normalised.

In [ ]:
df_norm.boxplot(column=columns, by=['Group'], figsize=(25, 25))
plt.savefig('full_metric_boxplots.png')
plt.show()

We choose to use LDA to use as our estimator. Feature ranking with recursive feature elimination and cross-validation (RFECV) can be used to reduce the number of metrics in our dataset. We are then left with a metric selection containing the best number of features.  

In [ ]:
def feature_selection(database, labels, n_splits=6, n_repeats=8):
            
    print("Total number of raw metrics = {}".format(database.shape[1]))
    X = database.values
    y = labels.values
    
    kfold = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=None)    
    lda = LinearDiscriminantAnalysis(solver='eigen', store_covariance=True)
    rfecv = RFECV(estimator=lda, step=1, cv=kfold, scoring='accuracy')
    
    rfecv.fit(X, y)

    print("Optimal number of features : %d" % rfecv.n_features_)    
    print("Features selected: ", database.loc[:, rfecv.support_].columns)
    print("Features dropped: ", database.loc[:, ~rfecv.support_].columns)
    
    # Drops features
    features = database.loc[:, rfecv.support_]

    text_size = 14
    mpl.rc('xtick', labelsize=text_size) 
    mpl.rc('ytick', labelsize=text_size) 
    mpl.rcParams.update({'font.size': text_size})
    
    # Plot number of features VS. cross-validation scores
    plt.figure(0, figsize=(8,6))
    plt.xlabel("Number of features selected")
    plt.ylabel("Cross validation score (accuracy %)")
    plt.plot(range(1, len(rfecv.grid_scores_) + 1), 100 * rfecv.grid_scores_)
    plt.savefig(f'cv_scores_lda.png')
    plt.show()
    
    return features

In [ ]:
labels = df_norm['Label'].copy()

df_full = df_norm.drop(['Group', 'Label'], axis=1)

df_red = feature_selection(df_full, labels)                            

Performing Linear Discriminant Analysis (LDA) on image metric database using labels supplied by medics.

In [ ]:
def lda_analysis(database, labels, tick_labels=None, n_splits=6, n_repeats=10, tag=''):
    
    X = database.values
    y = labels.values
    
    kfold = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=None)
    lda = LinearDiscriminantAnalysis(solver='eigen', store_covariance=True)
    
    scores = []
    binary_scores = []
    
    for train_index, test_index in kfold.split(X, y):
        X_train, y_train = X[train_index], y[train_index]
        X_test, y_test = X[test_index], y[test_index]
        
        lda.fit(X_train, y_train)
        y_predict = lda.predict(X_test)

        roc_y = np.where(y_test == 1, 1, 0)
        roc_y_test = np.where(y_predict == 1, 1, 0)
        
        binary_scores.append(accuracy_score(roc_y, roc_y_test))
        scores.append(accuracy_score(y_test, y_predict))
    
    print("Average cross validation score for all classes: ", np.mean(scores), "+= ", np.std(scores))
    print("Average cross validation score for Normal vs other classes: ", np.mean(binary_scores), "+= ", np.std(binary_scores))
    
    # Plotting the 
    X_plot_train = lda.transform(X_train)
    X_plot_test = lda.transform(X_test)
    
    plt.figure(0)
    fig, ax, cb = scatter(X_plot_train, y_train, np.ones(y_train.shape))
    fig, ax, cb = scatter(X_plot_test, y_test, np.ones(y_test.shape),
                          ellipse=False, marker='x', alpha=0.6, fig=fig, ax=ax, cb=cb)
    cb.set_ticklabels(tick_labels)
    #plt.axis('off')
    #plt.axis([-5, 5, -5, 5])
    plt.tight_layout()
    plt.savefig(f'lda_{tag}.png')
    plt.show()
    
    fig = plt.figure(figsize=(10, 8))
    ax = plt.gca()
    
    im = ax.matshow(lda.coef_, interpolation='none', cmap='coolwarm')
    fig.colorbar(im)
    
    ax.set_xticks(np.arange(len(database.columns)))
    ax.set_xticklabels(list(database.columns))
    ax.set_yticks(np.arange(len(tick_labels)))
    ax.set_yticklabels(tick_labels)
    
    plt.setp([tick.label2 for tick in ax.xaxis.get_major_ticks()], rotation=45,
         ha="left", va="center",rotation_mode="anchor")
    
    plt.show()
    
    fig = plt.figure(figsize=(10, 8))
    ax = plt.gca()
    
    im = ax.matshow(lda.covariance_, interpolation='none', cmap='coolwarm')
    fig.colorbar(im)
    
    ax.set_xticks(np.arange(len(database.columns)))
    ax.set_xticklabels(list(database.columns))
    ax.set_yticks(np.arange(len(database.columns)))
    ax.set_yticklabels(list(database.columns))
    
    plt.setp([tick.label2 for tick in ax.xaxis.get_major_ticks()], rotation=45,
         ha="left", va="center",rotation_mode="anchor")
    
    plt.show()

In [ ]:
tick_labels = labels # Change this if you want to use different labels for the figures (i.e. use English spellings)

print('Using full set of PyFibre Metrics')
lda_analysis(df_full, labels, tick_labels=tick_labels, tag='full')

print('Using reduced set of PyFibre Metrics')
lda_analysis(df_red, labels, tick_labels=tick_labels, tag='red')

Next we perform PCA analysis on the databases.

In [ ]:
def pca_analysis(database, labels, tick_labels, per_var=0.9, supervised=False):

    (X_train, X_test, 
         y_train, y_test) = train_test_split(database, labels, train_size=0.8)
    
    pca = PCA(n_components=per_var, whiten=True, svd_solver='full')
    
    print("Total number of raw metrics = {}".format(database.shape[1]))
    print(database.columns)
    
    print("Training PCA on set of {} images".format(X_train.shape[0]))
    
    X = pca.fit_transform(X_train)

    plt.plot(pca.explained_variance_ratio_.cumsum())
    plt.show()
    
    n_components = len(pca.explained_variance_)
    print("PCA components retained = {}".format(n_components))

    y = np.array(labels)

    plt.figure(1)
    fig, cb = scatter(X, y_train, np.ones(y_train.shape))
    cb.set_ticklabels(tick_labels)
    plt.axis('off')
    plt.axis([-5, 5, -5, 5])
    plt.tight_layout()
    plt.show()
    
    print("Performing PCA transform on test set of {} images".format(X_test.shape[0]))

    plt.figure(1)
    fig, cb = scatter(pca.transform(X_test), y_test, np.ones(y_test.shape))
    cb.set_ticklabels(groups)
    plt.axis('off')
    plt.axis([-5, 5, -5, 5])
    plt.tight_layout()
    plt.show()
    
    """
    for i, label in enumerate(tick_labels):
        plt.figure()
        fig, cb = scatter(X, y, np.where(y == i + 1, 1, 0))
        cb.set_ticklabels(tick_labels)
        plt.axis('off')
        plt.tight_layout()
        plt.show()
    ""

In [ ]:
print('Using full set of PyFibre Metrics')
pca_analysis(df_full, labels, tick_labels=tick_labels)

print('Using reduced set of PyFibre Metrics')
pca_analysis(df_red, labels, tick_labels=tick_labels)